In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [85]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    #Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [86]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [87]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

In [88]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [89]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
encoder_output = word_embeddings

for i in range(1):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 1s - loss: 3.2522 - val_loss: 3.2352 - 1s/epoch - 1s/step
Epoch 2/2000
1/1 - 0s - loss: 3.2310 - val_loss: 3.2201 - 37ms/epoch - 37ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.2127 - val_loss: 3.2063 - 36ms/epoch - 36ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.1960 - val_loss: 3.1936 - 38ms/epoch - 38ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.1808 - val_loss: 3.1818 - 40ms/epoch - 40ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.1669 - val_loss: 3.1708 - 40ms/epoch - 40ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.1541 - val_loss: 3.1606 - 37ms/epoch - 37ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.1423 - val_loss: 3.1512 - 37ms/epoch - 37ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.1314 - val_loss: 3.1425 - 37ms/epoch - 37ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.1211 - val_loss: 3.1343 - 39ms/epoch - 39ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.1116 - val_loss: 3.1267 - 39ms/epoch - 39ms/step
Epoch 12/2000
1/1 - 0s - loss: 3.1027 - val_loss: 3.1197 - 38ms/epoch - 38ms/step
Epoch 13/2000
1/1 - 0s - loss

1/1 - 0s - loss: 2.9090 - val_loss: 2.9591 - 36ms/epoch - 36ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.9079 - val_loss: 2.9584 - 37ms/epoch - 37ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.9068 - val_loss: 2.9576 - 39ms/epoch - 39ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.9057 - val_loss: 2.9568 - 41ms/epoch - 41ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.9046 - val_loss: 2.9561 - 37ms/epoch - 37ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.9035 - val_loss: 2.9554 - 36ms/epoch - 36ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.9025 - val_loss: 2.9546 - 37ms/epoch - 37ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.9014 - val_loss: 2.9539 - 41ms/epoch - 41ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.9004 - val_loss: 2.9532 - 39ms/epoch - 39ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.8993 - val_loss: 2.9526 - 38ms/epoch - 38ms/step
Epoch 111/2000
1/1 - 0s - loss: 2.8983 - val_loss: 2.9519 - 37ms/epoch - 37ms/step
Epoch 112/2000
1/1 - 0s - loss: 2.8973 - val_loss: 2.9512 - 36ms/epoch - 36ms/step
Epoch 113/2000
1/1 

In [90]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [91]:
(np.mean(acc), np.mean(prob))

(0.295, 0.097396016)

In [92]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
encoder_output = word_embeddings

for i in range(1):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 1s - loss: 3.6705 - val_loss: 3.4535 - 1s/epoch - 1s/step
Epoch 2/2000
1/1 - 0s - loss: 3.4047 - val_loss: 3.3180 - 80ms/epoch - 80ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.2608 - val_loss: 3.2204 - 76ms/epoch - 76ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.1572 - val_loss: 3.1491 - 72ms/epoch - 72ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.0823 - val_loss: 3.0953 - 79ms/epoch - 79ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.0266 - val_loss: 3.0547 - 76ms/epoch - 76ms/step
Epoch 7/2000
1/1 - 0s - loss: 2.9849 - val_loss: 3.0265 - 74ms/epoch - 74ms/step
Epoch 8/2000
1/1 - 0s - loss: 2.9558 - val_loss: 3.0070 - 74ms/epoch - 74ms/step
Epoch 9/2000
1/1 - 0s - loss: 2.9353 - val_loss: 2.9918 - 82ms/epoch - 82ms/step
Epoch 10/2000
1/1 - 0s - loss: 2.9187 - val_loss: 2.9788 - 73ms/epoch - 73ms/step
Epoch 11/2000
1/1 - 0s - loss: 2.9038 - val_loss: 2.9677 - 75ms/epoch - 75ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.8905 - val_loss: 2.9585 - 75ms/epoch - 75ms/step
Epoch 13/2000
1/1 - 0s - loss

In [93]:
x_masked_train

array([[ 0, 10,  2,  0],
       [18, 16, 19, 18],
       [16, 14, 13, 16],
       ...,
       [ 1, 11,  0,  1],
       [18, 11, 16, 18],
       [ 9,  1, 19,  9]])

In [94]:
y_masked_labels_train

array([[10,  2,  0],
       [16, 19, 18],
       [14, 13, 16],
       ...,
       [11,  0,  1],
       [11, 16, 18],
       [ 1, 19,  9]])

In [95]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [96]:
(np.mean(acc), np.mean(prob))

(0.481, 0.17966855)

In [97]:
x_test_subset

array([[15,  0, 13, 15],
       [17, 11,  9, 17],
       [10,  0,  3, 10],
       ...,
       [12,  2,  7, 12],
       [ 9, 16,  3,  9],
       [12,  6, 18, 12]])

In [98]:
temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array([5, 0, 2, 5]).reshape(1,len(sentence_number)))
temp = temp[:,-1,:]
temp

array([[0.18322888, 0.04183327, 0.00042343, 0.04988674, 0.02165989,
        0.23305799, 0.051962  , 0.03700745, 0.05091651, 0.0318071 ,
        0.02159526, 0.02821101, 0.02173642, 0.02830644, 0.02710385,
        0.05953328, 0.02446126, 0.02872639, 0.04173755, 0.01680526]],
      dtype=float32)

In [99]:
temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array([0, 5, 2, 0]).reshape(1,len(sentence_number)))
temp = temp[:,-1,:]
temp

array([[0.18322888, 0.04183327, 0.00042343, 0.04988674, 0.02165989,
        0.23305799, 0.051962  , 0.03700745, 0.05091651, 0.0318071 ,
        0.02159526, 0.02821101, 0.02173642, 0.02830644, 0.02710385,
        0.05953328, 0.02446126, 0.02872639, 0.04173755, 0.01680526]],
      dtype=float32)